In [1]:
import requests as re
from bs4 import BeautifulSoup as bs

In [7]:
data = re.get("https://www.erowid.org/experiences/exp.cgi?S=0&C=1&ShowViews=0&Cellar=0&Start=0&Max=1000")

In [8]:
print(data.text)

<HTML>
<head>
<title> Reports - General : Erowid Experience Vaults</title>
<META NAME="description" CONTENT="Erowid Experience Vaults: An Experience">
<META NAME="keywords" CONTENT="Experience Report Vaults, trip reports, stories, descriptions">
<link href="/includes/general_default.css" rel="stylesheet" type="text/css">
<link href="includes/exp.css" rel="stylesheet" type="text/css">
<!-- Sperowider <noindex/> -->
</head>
<BODY BGCOLOR="#000000" TEXT="#999977" LINK="#7777AA" VLINK="#999999" ALINK="#008080" >
<TABLE BORDER=0 CELLPADDING=0 CELLSPACING=0 ALIGN=CENTER><TR><TD>
<IMG SRC="/experiences/images/exp_header_blk.gif" WIDTH=707 HEIGHT=76 BORDER=0 USEMAP="#exp_header">
</TD></TR><TR HEIGHT=3><TD></TD></TR><TR><TD ALIGN=CENTER>
<a href="/donations/donations.shtml"><font face="helvetica,arial" size="2" color="#FFBB22"><b>
Think people should have access to truthful info about drugs?<br/>
We need your help to make that happen.  Join Erowid today!
</b></font></a><br/>

</TD></TR></TABLE